In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
# from torch_points3d.datasets.segmentation.scannet import ScannetDataset, IGNORE_LABEL
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM, IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/scannet'  # ENGIE

# XYZ+RGB
overrides = [
#     'model_name=Res16UNet34-L4-early',
    
#     'model_name=Res16UNet34-L4-early-ade20k-interpolate',
    'model_name=Res16UNet34-PointPyramid-early-ade20k-interpolate',
    
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-20/18-43-13'}",
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-24/23-12-40'}",
    
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-12-29/18-02-24'}",
    f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-12-11/15-51-10'}",
    
    'voting_runs=1',
    'tracker_options.full_res=True',
#     'tracker_options.make_submission=True',
    'precompute_multi_scale=False',
    'num_workers=4',
    'batch_size=4',
    f'cuda={I_GPU}',
    'weight_name=miou',
#     'weight_name=last',
]

# # XYZRGB
# overrides = [
#     'model_name=Res16UNet34',
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-11-05/23-21-24'}",  # 3cm
#     'voting_runs=1',
#     'tracker_options.full_res=True',
#  #   'tracker_options.make_submission=True',
#     'precompute_multi_scale=False',
#     'num_workers=4',
#     'batch_size=4',
#     f'cuda={I_GPU}',
# #     'weight_name=miou',
#     'weight_name=last',
# ]

cfg = hydra_read(overrides, config_name='eval')

OmegaConf.set_struct(cfg, False)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

In [ ]:
from torch_points3d.trainer import Trainer

trainer = Trainer(cfg)

In [ ]:
test_transform = trainer._dataset.test_dataset[0].transform
test_transform_image = trainer._dataset.test_dataset[0].transform_image
val_transform = trainer._dataset.val_dataset.transform
val_transform_image = trainer._dataset.val_dataset.transform_image

In [ ]:
# Update the val and test transforms to match train transforms for inference-time augmentation
trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image

trainer._dataset.val_dataset.transform = trainer._dataset.train_dataset.transform
trainer._dataset.val_dataset.transform_image = trainer._dataset.train_dataset.transform_image

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000
trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 / 4)
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 / 4)

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.02
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.8, 1.2]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.02
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.8, 1.2]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.8, 1.2]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.8, 1.2]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.03
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.6, 1.4]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.6, 1.4]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.6, 1.4]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.03
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.6, 1.4]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.6, 1.4]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.6, 1.4]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.01
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.9, 1.1]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.9, 1.1]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.9, 1.1]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.01
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.9, 1.1]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.9, 1.1]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.9, 1.1]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.005
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.95, 1.05]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.95, 1.05]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.95, 1.05]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.005
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.95, 1.05]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.95, 1.05]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.95, 1.05]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.9, 1.1]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.01
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.95, 1.05]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 4)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.001
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.99, 1.01]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.2
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.1, 1.9]

In [ ]:
# Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].mean = [0, 0, 0]
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].std = [1, 1, 1]

In [ ]:
# trainer._cfg.tracker_options.full_res = False

In [ ]:
trainer._cfg.voting_runs = 3

In [ ]:
# Validation inference
trainer.eval(stage_name="val")

In [ ]:
# # Test inference
# trainer.eval(stage_name="test")

In [ ]:
# torch.save(trainer._tracker._full_confusion_matrix, '/raid/dataset/pointcloud/data/scannet/results/xyz+rgb_3cm_val.pt')

In [ ]:
trainer._tracker.get_metrics(True)

# Inference visualizations for ScanNet

In [ ]:
trainer._dataset.val_dataset.transform.transforms = val_transform.transforms[:-1]
trainer._dataset.val_dataset.transform_image = val_transform_image

In [ ]:
# Inference for Baseline
from torch_geometric.data import Batch

model = trainer._model.eval().cuda()

with torch.no_grad():
    pred_baseline = None
    for i in range(1):
        data = Batch.from_data_list([trainer._dataset.val_dataset[0]])
        model.set_input(data, model.device)
        model(data)
        if pred_baseline is None:
            pred_baseline = model.output.cpu()
        else:
            pred_baseline += model.output.cpu()
    pred_baseline = pred_baseline.argmax(1)

# Save the prediction for backup
# torch.save(pred_baseline, '/raid/dataset/pointcloud/data/scannet/results/inferences_for_visualization/baseline_y.pt')

In [ ]:
# Inference for Ours
from torch_points3d.core.multimodal.data import MMData, MMBatch

model = trainer._model.eval().cuda()

with torch.no_grad():
    pred_ours = None
    for i in range(1):
        mm_data = MMBatch.from_mm_data_list([trainer._dataset.val_dataset[0]])
        model.set_input(mm_data, model.device)
        model(mm_data)
        if pred_ours is None:
            pred_ours = model.output.cpu()
        else:
            pred_ours += model.output.cpu()
    pred_ours = pred_ours.argmax(1)

In [ ]:
CLASS_COLORS

In [ ]:
CLASS_NAMES

In [ ]:
from torch_points3d.datasets.segmentation.scannet import VALID_CLASS_IDS, SCANNET_COLOR_MAP, CLASS_LABELS, NUM_CLASSES, IGNORE_LABEL, CLASS_COLORS, CLASS_NAMES
from torch_points3d.visualization.multimodal_data import visualize_mm_data

def remap_scannet_labels(semantic_label, valid_class_idx=VALID_CLASS_IDS, donotcare_class_ids=[]):
    """Remaps labels to [0 ; num_labels -1]."""
    new_labels = semantic_label.clone()
    mapping_dict = {idx: i for i, idx in enumerate(valid_class_idx)}
    for idx in range(NUM_CLASSES):
        if idx not in mapping_dict:
            mapping_dict[idx] = IGNORE_LABEL
    for idx in donotcare_class_ids:
        mapping_dict[idx] = IGNORE_LABEL
    for source, target in mapping_dict.items():
        mask = semantic_label == source
        new_labels[mask] = target

    broken_labels = new_labels >= len(valid_class_idx)
    new_labels[broken_labels] = IGNORE_LABEL

    return new_labels

In [ ]:
# Build the MMData objects
mm_data_gt = mm_data.clone()
mm_data_gt.modalities['image'] = mm_data_gt.modalities['image'][0][[]]

mm_data_ours = mm_data.clone()
mm_data_ours.modalities['image'] = mm_data_ours.modalities['image'][0][[]]
mm_data_ours.data.y = pred_ours

# mm_data_baseline = mm_data.clone()
# mm_data_baseline.modalities['image'] = mm_data_baseline.modalities['image'][0][[]]
# mm_data_baseline.data.y = pred_baseline

In [ ]:
from torch_points3d.core.multimodal.image import ImageData, SameSettingImageData
mm_data_gt = torch.load('/raid/dataset/pointcloud/data/scannet/results/inferences_for_visualization/gt.pt')
mm_data_gt.modalities['image'] = ImageData([SameSettingImageData()])
visualize_mm_data(mm_data_gt, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, show_2d=False, figsize=1600, pointsize=4, voxel=0.05)

In [ ]:
mm_data_ours = torch.load('/raid/dataset/pointcloud/data/scannet/results/inferences_for_visualization/ours.pt')
mm_data_ours.modalities['image'] = ImageData([SameSettingImageData()])
visualize_mm_data(mm_data_ours, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, show_2d=False, figsize=1600, pointsize=4, voxel=0.05)

In [ ]:
mm_data_baseline = torch.load('/raid/dataset/pointcloud/data/scannet/results/inferences_for_visualization/baseline.pt')
mm_data_baseline.modalities['image'] = ImageData([SameSettingImageData()])
visualize_mm_data(mm_data_baseline, class_names=CLASS_NAMES, class_colors=CLASS_COLORS, show_2d=False, figsize=1600, pointsize=4, voxel=0.05)

In [ ]:
# vote=3 - x2 credit - train aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - even softer 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - even softer 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - softer 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=5 - ÷4 credit - harder 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - harder 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - milder 2D aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=5 - ÷4 credit - train aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit - train aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit AGAIN
trainer._tracker.get_metrics(True)

In [ ]:
# vote=5 - ÷4 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷6 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷8 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷2 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - x2 credit
trainer._tracker.get_metrics(True)

In [ ]:
# vote=5 - no aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - no aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=1 - no aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - softer aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=1 - aug
trainer._tracker.get_metrics(True)

# XYZ+RGB 5cm Val

In [ ]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

In [ ]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/scannet/results/xyz+rgb_5cm_val.pt')

In [ ]:
import torch

# cm = torch.load('/raid/dataset/pointcloud/data/scannet/results/sparseconv3d_3cm.pt')
cm = torch.load('/raid/dataset/pointcloud/data/scannet/results/xyz+rgb_3cm_val.pt')
cm.get_average_intersection_union()

In [ ]:
' & '.join([str(x) for x in np.round((cm.get_intersection_union_per_class()[0]) * 100, 1)])